# WasteNet

## Setup

In [ ]:
# Colab
!rm -r smart-cities-drl
!git clone https://github.com/eescriba/smart-cities-drl
!cd smart-cities-drl/ && pip install -e .

import sys
sys.path.insert(0,'./smart-cities-drl/src/')

# Local
# !pip install -e ..
# import sys
# sys.path.insert(0,'../src/')

In [26]:
import json
import shutil
import random

import gym
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune import run, sample_from
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
from ray.tune.schedulers import PopulationBasedTraining
import tensorflow as tf

from wastenet.env import WasteNetEnv

In [32]:
register_env("WasteNet-v0", lambda config: WasteNetEnv(config))
num_gpus = len(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus)

Num GPUs Available:  0


In [33]:
ray.shutdown()
ray.init(ignore_reinit_error=True, num_gpus=num_gpus)

2021-05-23 18:01:13,196	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.100.180',
 'raylet_ip_address': '192.168.100.180',
 'redis_address': '192.168.100.180:6379',
 'object_store_address': '/tmp/ray/session_2021-05-23_18-01-10_132184_46580/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-23_18-01-10_132184_46580/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-23_18-01-10_132184_46580',
 'metrics_export_port': 47135,
 'node_id': '54ad67fc224d4ee2d2b27f9b809a3100bbd8caf4ff2177a84199c3f2'}

## Proximal Policy Optimization (PPO)

### Population Based Training (PBT)

In [ ]:

# Postprocess the perturbed config to ensure it's still valid used if PBT.
def explore(config):
    # ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["sgd_minibatch_size"] * 2:
        config["train_batch_size"] = config["sgd_minibatch_size"] * 2
    # ensure we run at least one sgd iter
    if config["num_sgd_iter"] < 1:
        config["num_sgd_iter"] = 1
    return config

pbt = PopulationBasedTraining(
    time_attr="time_total_s",
    perturbation_interval=120,
    resample_probability=0.25,
    metric="episode_reward_mean",
    mode="max",
    # Specifies the mutations of these hyperparams
    hyperparam_mutations={
        "lambda": lambda: random.uniform(0.7, 1.0),
        "clip_param": lambda: random.uniform(0.01, 0.5),
        "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
        "num_sgd_iter": lambda: random.randint(1, 30),
        "sgd_minibatch_size": lambda: random.randint(128, 16384),
        "train_batch_size": lambda: random.randint(2000, 160000),
    },
    custom_explore_fn=explore)

### Tune hyperparameters

In [ ]:
analysis = run(
        'PPO',
        name="wastenet_ppo",
        local_dir="./ray_results",
        resume=True,
        scheduler=pbt,
        num_samples=8,
        config={
            "env": WasteNetEnv,
            "seed": 123,
            "num_gpus": num_gpus,
            "num_workers": 1,
            "observation_filter": "MeanStdFilter",
            "model": {
                # "fcnet_hiddens": [
                #     32,
                #     32
                # ],
                "free_log_std": True
            },
            "num_sgd_iter": 10,
            "sgd_minibatch_size": 128,
            "lambda": sample_from(lambda spec: random.uniform(0.9, 1.0)),
            "clip_param": sample_from(lambda spec: random.uniform(0.1, 0.5)),
            "lr": sample_from(lambda spec: random.uniform(1e-3, 1e-5)),
            "train_batch_size": sample_from(
                lambda spec: random.randint(1000, 60000))
        })

best_config=analysis.best_config
print("best hyperparameters: ", best_config)

### Training

In [ ]:
best_config = {
    "observation_filter": "MeanStdFilter",
    "model": {"free_log_std": True},
    "num_sgd_iter": 10,
    "sgd_minibatch_size": 128,
    "lambda": 0.731396,
    "clip_param": 0.317651,
    "lr": 5e-05,
    "train_batch_size": 18812,
}

In [ ]:

config = best_config
config["num_gpus"] = num_gpus

agent = ppo.PPOTrainer(config, env=WasteNetEnv)

N_ITER = 1000

for n in range(N_ITER):
    result = agent.train()
    print(pretty_print(result))
    if i % 10 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)

In [ ]:
policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

## Evaluation

In [ ]:
!rllib rollout \
  --run PPO  \
  --env WasteNet-v0  \
  --steps 1000 \
  --config best_config

In [ ]:
obs = env.reset()
done = False
episode_reward = 0
sum_reward = 0
n_step = 1000
for step in range(n_step):
    action = agent.compute_action(obs)
    print(action)
    state, reward, done, info = env.step(action)
    print(state, reward, done, info)
    sum_reward += reward
    if done:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [14]:
%load_ext tensorboard 
%tensorboard --logdir="/content/ray_results/wastenet_ppo"

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C
